In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql.functions import  *
from datetime import date
from pyspark.sql.types import * 
from pyspark.sql import Window
from datetime import datetime

In [2]:
spark = (
    SparkSession.builder
    .master("local")
    .appName("Exploring Joins")
    .config("spark.some.config.option", "some-value")
    .getOrCreate()
)

In [5]:
df = spark.read.csv('pets.csv',header = True)

In [6]:
df.show()

+---+--------+--------+-------------------+---+-----+------+
| id|breed_id|nickname|           birthday|age|color|weight|
+---+--------+--------+-------------------+---+-----+------+
|  1|       1|    King|2014-11-22 12:30:31|  5|brown|  10.0|
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|   3.4|
|  4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|
+---+--------+--------+-------------------+---+-----+------+



In [8]:
df.toPandas()

,id,breed_id,nickname,birthday,age,color,weight
0,1,1,King,2014-11-22 12:30:31,5,brown,10.0
1,2,3,Argus,2016-11-22 10:05:10,10,None,5.5
2,3,1,Chewie,2016-11-22 10:05:10,15,None,12
3,3,2,Maple,2018-11-22 10:05:10,17,white,3.4
4,4,2,None,2019-01-01 10:05:10,13,None,10


In [9]:
df.collect()

[Row(id='1', breed_id='1', nickname='King', birthday='2014-11-22 12:30:31', age='5', color='brown', weight='10.0'),
 Row(id='2', breed_id='3', nickname='Argus', birthday='2016-11-22 10:05:10', age='10', color=None, weight='5.5'),
 Row(id='3', breed_id='1', nickname='Chewie', birthday='2016-11-22 10:05:10', age='15', color=None, weight='12'),
 Row(id='3', breed_id='2', nickname='Maple', birthday='2018-11-22 10:05:10', age='17', color='white', weight='3.4'),
 Row(id='4', breed_id='2', nickname=None, birthday='2019-01-01 10:05:10', age='13', color=None, weight='10')]

In [10]:
df.head()

Row(id='1', breed_id='1', nickname='King', birthday='2014-11-22 12:30:31', age='5', color='brown', weight='10.0')

In [11]:
df.select('id','breed_id').show()

+---+--------+
| id|breed_id|
+---+--------+
|  1|       1|
|  2|       3|
|  3|       1|
|  3|       2|
|  4|       2|
+---+--------+



In [12]:
df.drop('id','breed_id').show()

+--------+-------------------+---+-----+------+
|nickname|           birthday|age|color|weight|
+--------+-------------------+---+-----+------+
|    King|2014-11-22 12:30:31|  5|brown|  10.0|
|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|
|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|
|   Maple|2018-11-22 10:05:10| 17|white|   3.4|
|    NULL|2019-01-01 10:05:10| 13| NULL|    10|
+--------+-------------------+---+-----+------+



In [14]:
df.withColumn('nick_named_copy',col('nickname')).withColumn('nick_name_capital',upper(col('nickname'))).show()

+---+--------+--------+-------------------+---+-----+------+---------------+-----------------+
| id|breed_id|nickname|           birthday|age|color|weight|nick_named_copy|nick_name_capital|
+---+--------+--------+-------------------+---+-----+------+---------------+-----------------+
|  1|       1|    King|2014-11-22 12:30:31|  5|brown|  10.0|           King|             KING|
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|          Argus|            ARGUS|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|         Chewie|           CHEWIE|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|   3.4|          Maple|            MAPLE|
|  4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|           NULL|             NULL|
+---+--------+--------+-------------------+---+-----+------+---------------+-----------------+



In [15]:
df.withColumnRenamed('id','pet_id').show()

+------+--------+--------+-------------------+---+-----+------+
|pet_id|breed_id|nickname|           birthday|age|color|weight|
+------+--------+--------+-------------------+---+-----+------+
|     1|       1|    King|2014-11-22 12:30:31|  5|brown|  10.0|
|     2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|
|     3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|
|     3|       2|   Maple|2018-11-22 10:05:10| 17|white|   3.4|
|     4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|
+------+--------+--------+-------------------+---+-----+------+



In [17]:
df.withColumn('dates',lit(date.today())).show()

+---+--------+--------+-------------------+---+-----+------+----------+
| id|breed_id|nickname|           birthday|age|color|weight|     dates|
+---+--------+--------+-------------------+---+-----+------+----------+
|  1|       1|    King|2014-11-22 12:30:31|  5|brown|  10.0|2024-06-26|
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|2024-06-26|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|2024-06-26|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|   3.4|2024-06-26|
|  4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|2024-06-26|
+---+--------+--------+-------------------+---+-----+------+----------+



In [18]:
df.withColumn('random_constant',lit(10)).show()

+---+--------+--------+-------------------+---+-----+------+---------------+
| id|breed_id|nickname|           birthday|age|color|weight|random_constant|
+---+--------+--------+-------------------+---+-----+------+---------------+
|  1|       1|    King|2014-11-22 12:30:31|  5|brown|  10.0|             10|
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|             10|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|             10|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|   3.4|             10|
|  4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|             10|
+---+--------+--------+-------------------+---+-----+------+---------------+



In [20]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- breed_id: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- age: string (nullable = true)
 |-- color: string (nullable = true)
 |-- weight: string (nullable = true)



In [28]:
df.withColumn('birthday',col('birthday').cast('date')).withColumn('age',col('age').cast('integer')).printSchema()

root
 |-- id: string (nullable = true)
 |-- breed_id: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- birthday: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- color: string (nullable = true)
 |-- weight: string (nullable = true)



In [29]:
df.withColumn('weight',col('weight').cast('integer')).show()

+---+--------+--------+-------------------+---+-----+------+
| id|breed_id|nickname|           birthday|age|color|weight|
+---+--------+--------+-------------------+---+-----+------+
|  1|       1|    King|2014-11-22 12:30:31|  5|brown|    10|
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|     5|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|     3|
|  4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|
+---+--------+--------+-------------------+---+-----+------+



In [30]:
df.where((col('age')>5) & (col('weight')<=5)).show()

+---+--------+--------+-------------------+---+-----+------+
| id|breed_id|nickname|           birthday|age|color|weight|
+---+--------+--------+-------------------+---+-----+------+
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|   3.4|
+---+--------+--------+-------------------+---+-----+------+



In [31]:
df.filter((col('age')>5) & (col('weight')<=5)).show()

+---+--------+--------+-------------------+---+-----+------+
| id|breed_id|nickname|           birthday|age|color|weight|
+---+--------+--------+-------------------+---+-----+------+
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|   3.4|
+---+--------+--------+-------------------+---+-----+------+



In [32]:
df.withColumn('result',col('color')!='white').show()

+---+--------+--------+-------------------+---+-----+------+------+
| id|breed_id|nickname|           birthday|age|color|weight|result|
+---+--------+--------+-------------------+---+-----+------+------+
|  1|       1|    King|2014-11-22 12:30:31|  5|brown|  10.0|  true|
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|   5.5|  NULL|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|  NULL|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|   3.4| false|
|  4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|  NULL|
+---+--------+--------+-------------------+---+-----+------+------+



In [43]:
df = df.withColumn('age',col('age').cast('integer')).withColumn('weight',col('weight').cast('integer'))
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- breed_id: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- color: string (nullable = true)
 |-- weight: integer (nullable = true)



In [44]:
df.withColumn('maturity',when(col('age')<=5,'young').when((col('age')>5) & (col('age')<14),'middle_age').otherwise('old')).show()

+---+--------+--------+-------------------+---+-----+------+----------+
| id|breed_id|nickname|           birthday|age|color|weight|  maturity|
+---+--------+--------+-------------------+---+-----+------+----------+
|  1|       1|    King|2014-11-22 12:30:31|  5|brown|    10|     young|
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|     5|middle_age|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12|       old|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|     3|       old|
|  4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|middle_age|
+---+--------+--------+-------------------+---+-----+------+----------+



In [46]:
df.select('nickname').fillna('ngawang').show()

+--------+
|nickname|
+--------+
|    King|
|   Argus|
|  Chewie|
|   Maple|
| ngawang|
+--------+



In [48]:
df.fillna({'nickname':'Ngawang','color':'Ngawanguu'}).show()

+---+--------+--------+-------------------+---+---------+------+
| id|breed_id|nickname|           birthday|age|    color|weight|
+---+--------+--------+-------------------+---+---------+------+
|  1|       1|    King|2014-11-22 12:30:31|  5|    brown|    10|
|  2|       3|   Argus|2016-11-22 10:05:10| 10|Ngawanguu|     5|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15|Ngawanguu|    12|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|    white|     3|
|  4|       2| Ngawang|2019-01-01 10:05:10| 13|Ngawanguu|    10|
+---+--------+--------+-------------------+---+---------+------+



In [51]:
df.withColumn('xyz',coalesce(col('color'),col('nickname'),lit('default'))).show()

+---+--------+--------+-------------------+---+-----+------+-------+
| id|breed_id|nickname|           birthday|age|color|weight|    xyz|
+---+--------+--------+-------------------+---+-----+------+-------+
|  1|       1|    King|2014-11-22 12:30:31|  5|brown|    10|  brown|
|  2|       3|   Argus|2016-11-22 10:05:10| 10| NULL|     5|  Argus|
|  3|       1|  Chewie|2016-11-22 10:05:10| 15| NULL|    12| Chewie|
|  3|       2|   Maple|2018-11-22 10:05:10| 17|white|     3|  white|
|  4|       2|    NULL|2019-01-01 10:05:10| 13| NULL|    10|default|
+---+--------+--------+-------------------+---+-----+------+-------+



In [53]:
df.groupBy('breed_id').agg({'*':'count','age':'sum','weight':'sum'}).show()

+--------+-----------+--------+--------+
|breed_id|sum(weight)|count(1)|sum(age)|
+--------+-----------+--------+--------+
|       3|          5|       1|      10|
|       1|         22|       2|      20|
|       2|         13|       2|      30|
+--------+-----------+--------+--------+



In [60]:
df.orderBy('birthday').groupBy('breed_id').agg(count("*").alias('count'),sum("age").alias('sum_age'),sum('weight').alias('sum_weight')).show()

+--------+-----+-------+----------+
|breed_id|count|sum_age|sum_weight|
+--------+-----+-------+----------+
|       3|    1|     10|         5|
|       1|    2|     20|        22|
|       2|    2|     30|        13|
+--------+-----+-------+----------+



In [93]:
window = Window.partitionBy('breed_id').orderBy("age")

In [94]:
pets = spark.createDataFrame(
    [
        (1, 1, datetime(2018, 1, 1, 1 ,1, 1), 'Bear', 5),
        (2, 1, datetime(2010, 1, 1, 1 ,1, 1), 'Chewie', 15),
        (3, 1, datetime(2015, 1, 1, 1 ,1, 1), 'Roger', 10),
        (4, 2, datetime(2015, 1, 2, 2 ,3, 4), 'Roger', 10),
        (5, 2, datetime(2015, 5, 6, 7 ,8, 9), 'Roger', 10),
    ], ['id', 'breed_id', 'birthday', 'nickname', 'age']
)


In [95]:
pets = pets.withColumn('row_number',row_number().over(window))

In [96]:
window2 = Window.partitionBy('breed_id').orderBy('row_number')

pets.withColumn("Sum_Age",sum('age').over(window2)).show()

+---+--------+-------------------+--------+---+----------+-------+
| id|breed_id|           birthday|nickname|age|row_number|Sum_Age|
+---+--------+-------------------+--------+---+----------+-------+
|  1|       1|2018-01-01 01:01:01|    Bear|  5|         1|      5|
|  3|       1|2015-01-01 01:01:01|   Roger| 10|         2|     15|
|  2|       1|2010-01-01 01:01:01|  Chewie| 15|         3|     30|
|  4|       2|2015-01-02 02:03:04|   Roger| 10|         1|     10|
|  5|       2|2015-05-06 07:08:09|   Roger| 10|         2|     20|
+---+--------+-------------------+--------+---+----------+-------+



In [106]:
window3 = Window.partitionBy('breed_id').orderBy('age')
pets.withColumn('Count_Id',dense_rank().over(window3)).show()

+---+--------+-------------------+--------+---+----------+--------+
| id|breed_id|           birthday|nickname|age|row_number|Count_Id|
+---+--------+-------------------+--------+---+----------+--------+
|  1|       1|2018-01-01 01:01:01|    Bear|  5|         1|       1|
|  3|       1|2015-01-01 01:01:01|   Roger| 10|         2|       2|
|  2|       1|2010-01-01 01:01:01|  Chewie| 15|         3|       3|
|  4|       2|2015-01-02 02:03:04|   Roger| 10|         1|       1|
|  5|       2|2015-05-06 07:08:09|   Roger| 10|         2|       1|
+---+--------+-------------------+--------+---+----------+--------+

